# Choose your own adventure story

### You are about to embark on a choosen your own adventure story with none other than Donal Trump! You have the option to inhabit the world of Alice in Wonderland, Sherlock, or Ulysses. As you move throughout the story, think about your next decision carefully...danger awaits around every turn!

> Stories
- Sherlock
- Alice in Wonderland
- Ulysses

#### Resources
> Trump speeches https://github.com/ryanmcdermott/trump-speeches

> TFIDF http://stackoverflow.com/questions/12118720/python-tf-idf-cosine-to-find-document-similarity


In [23]:
from textblob import TextBlob
from collections import Counter
from collections import defaultdict
import nltk
from functools import reduce
import operator
import numpy as np
import re
from spacy.en import English
## The sentence generator module contains a trigram model for sentence generation
from sentence_generator import prob_distribution,cond_prob_bigram,cond_prob_trigram,sentence_generate
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from story_chunks import place_of_story

In [56]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
%autoreload 2

In [12]:
# First, we need to decide which world the user would like to inhabit
story_names = ['Grimms fairy tales','Alice in Wonderland','Ulysses']
while True:
    story = input("Which story would you like to read? You can choose Sherlock, Alice in Wonderland, or Ulysses.")
    if story not in story_names:
        print('Please spell the name correctly.')
    if story in story_names:
        break

Which story would you like to read? You can choose Sherlock, Alice in Wonderland, or Ulysses.Alice in Wonderland


In [103]:
## Open Trump speeches to predict sentences off of
with open("speeches.txt") as f:
        text_trump = f.read()
#Clean trump text
text= re.sub(r'[-!@#$%^&*()\n\]]+',' ',text_trump)
text = text.lower()
#Token the Trump words, count the frequency of each token, and create a unigram, bigram, and trigram dictionary
blob_text = TextBlob(text)
text_tokens = blob_text.tokens
text_counter_unigram_trump = Counter(text_tokens)
text_counter_bigram_trump = Counter(nltk.bigrams(text_tokens))
text_counter_trigram_trump = Counter(nltk.trigrams(text_tokens))
#Probability distributions for different text combinations
#This takes ~10 minutes to run
prob_dist_unigram_trump = prob_distribution(text_counter_unigram)
prob_dist_bigram_trump = prob_distribution(text_counter_bigram)
prob_dist_trigram_trump = prob_distribution(text_counter_trigram)

In [46]:
## Open the appropriate story based upon the user input
if story =='Alice in Wonderland':
    with open("alice_in_wonderland_copy.txt") as f:
        text_s = f.read()
elif story =='Grimms fairy tales':
    with open("grimms_fairy_tales.txt") as f:
        text_s = f.read()
elif story =='Ulysses':
    with open("ulysses_copy.txt") as f:
        text_s = f.read()
#Clean the text. Remove all punctuation except for periods, question marks, and exclamation points.
punctuation ='[ \ * \ [ - _ , : ; @ # $ % ^ & * ( ) \]  \n \']'.split(' ')
text_t = re.sub(r'[-!@#$%^&*()\n]',' ',text_s)

#Token the words, count the frequency of each token, and create a unigram, bigram, and trigram dictionary
# blob_text_story = TextBlob(text_t)
# text_tokens_story = blob_text_story.tokens
# text_counter_unigram_story = Counter(text_tokens_story)
# text_counter_bigram_story = Counter(nltk.bigrams(text_tokens_story))
# text_counter_trigram_story = Counter(nltk.trigrams(text_tokens_story))

# #Probability distributions for different text combinations
# #This takes ~10 minutes to run
# prob_dist_unigram_story = prob_distribution(text_counter_unigram_story)
# prob_dist_bigram_story = prob_distribution(text_counter_bigram_story)
# prob_dist_trigram_story = prob_distribution(text_counter_trigram_story)

In [45]:
text_t[:900]

"alice's adventures in wonderland  lewis carroll  the millennium fulcrum edition 3.0     chapter i. down the rabbit hole  alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought alice 'without pictures or conversations?'  so she was considering in her own mind  as well as she could, for the hot day made her feel very sleepy and stupid , whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her.  there was nothing so very remarkable in that; nor did alice think it so very much out of the way to hear the rabbit say to itself, 'oh dear  oh dear  i shall be late '  when she though"

In [47]:
#NER for the story characters and places - Entites {'GPE', 'CARDINAL', 'PRODUCT',
#'DATE', 'ORG', 'WORK_OF_ART', 'NORP', 'QUANTITY', 'TIME', 'PERSON', 'ORDINAL', 'MONEY', 'LOC', 'LANGUAGE', 'FAC'}
# takes ~ 2 minutes
nlp = English()
ner_story_tokens = nlp(text_t)
count=0
possible_entities = defaultdict(list)
for token in ner_story_tokens:
    if token.ent_type_ != "":
        if str(token) in possible_entities.values():
            pass
        else:
            if len(token)<3: ## check that it is an actual entity 
                pass
            else:
                possible_entities[token.ent_type_].append(str(token).lower())

In [48]:
## find the most common entities used in the story
people_counter = Counter(possible_entities['PERSON'])
place_counter = Counter(possible_entities['GPE'])
org_counter = Counter(possible_entities['ORG'])
language_counter = Counter(possible_entities['LANGUAGE'])
product_counter = Counter(possible_entities['PRODUCT'])

## We have a text generator for Trump and NER recognition for the story; now, move onto the actual adventure!

In [50]:
### need to keep track of the different entities used and in what order, # put in items here
#story - variable for story name
#Counters below
#people_counter = Counter(possible_entities['PERSON'])
# place_counter = Counter(possible_entities['GPE'])
# org_counter = Counter(possible_entities['ORG'])
# language_counter = Counter(possible_entities['LANGUAGE'])
# product_counter = Counter(possible_entities['PRODUCT'])
#items_chosen , choices of items made
protagonist=people_counter.most_common(1)[0][0].lower()
antagonist = people_counter.most_common(2)[1][0].lower()
place_one= place_counter.most_common(1)[0][0].lower()
place_two = place_counter.most_common(2)[1][0].lower()
language_one = language_counter.most_common(1)[0][0].lower()


#utility_item_chocen, funny_item_chosen, transportation_item_chosen
utility_items='pocketknife flashlight'.split(' ')
funny_items='half-eaten-sandwich broken-lightbulb'.split(' ')
transportation_items = 'skateboardbBike rollerblades'.split(' ')


In [ ]:
print('Welcome to a choose yur own adventure story!\n Below you will pick a story, and you will explore that world with a fellow campanion./n\
Throughout the story, you will have different options for next steps to take.\n \
Your goal is to choose the options that lead you safely on your adventure. \n \
However, be careful! Danger awaits around every turn.')

In [104]:


sentence_generate(prob_dist_unigram_trump,prob_dist_bigram_trump,prob_dist_trigram_trump,\
                  3,prob_dist_trigram_trump,prob_dist_bigram_trump,prob_dist_unigram_trump,\
                  text_counter_bigram_trump,text_counter_trigram_trump)

/Users/jonathanhilgart/DSCI6004-student/Final_Project/sentence_generator.py:55: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  generated_text.append(starting_chars[start_char_index])


'Rush hour when the great traders , one of the entire middle east . Our country is a terrific person . One of the blue and i shouldn’t say that . '

In [ ]:
###### The first section of the story   
print('You awake and look around "Hey, I think I recognize this place", you think to yourself. You realize \
that you are sitting down on something, you can not quite tell what, but you feel strangely awake.\
You turn to your right and see {} from {}. On your left, you see \
Donald Trump. You wonder where you are and how this all happened, but before you can think too much \
Trump turns to you and says '.format(protagonist,story),\
      sentence_generate(prob_dist_unigram_trump,prob_dist_bigram_trump,prob_dist_trigram_trump,3,\
                         prob_dist_trigram_trump,prob_dist_bigram_trump,prob_dist_unigram_trump),\
        'Well that was weird you think to yourself. Shaking it off, you turn to {} who says, \
        "Welcome to {}! We are excited to have you and your friend explore our \
        world. Before we start what do you want to bring with you on your \
        journey?'.format(protagonist,story),\
        "'Please pick one from each category,' instructes {}.".format(protagonist))
print()## keep track of the options chosen
while True:
    print()
    print(utility_items)
    utility_item_chosen = input("Which item would you like?")
    if utility_item_chosen in utility_items:
        break
    else:
        print('Please spell the word correctly without the quotes')
while True:
    print()
    print(funny_items)
    funny_item_chosen = input("Which of these would you like?")
    if funny_item_chosen in funny_items:
        break
    else:
        print('Please spell the word correctly without the quotes')
while True:
    print()
    print(transportation_items)
    trans_item_chosen = input('Which of these would you like?')
    if trans_item_chosen in transportation_items:
        break
    else:
        print('Please spell the word correctly without the quotes')
items_chosen =[utility_item_chosen,funny_item_chosen,trans_item_chosen]
while True:
    choice1 = input('Please write a sentence for what you would like to do next incorporating an item you have choosen.\
    If you want to see the items you have, type items.')  
    if choice1 =='items':
        print(utility_item_chosen,trans_item_chosen,funny_item_chosen)
    else:
        return choice1


### After each block of story, the sentence entered by the user will be picked by an IR system  for the next part of the story.

In [ ]:
antagonist = []
antagonist = people_counter.most_common(3)[0][0]
antagonist

In [ ]:
people_counter.most_common(2)[1][0]

In [ ]:
possible_entities['PERSON']

# TESTING

In [ ]:
#prob_dist_unigram_trump = prob_distribution(text_counter_unigram)
prob_dist_bigram_trump = prob_distribution(text_counter_bigram)
prob_dist_trigram_trump

In [14]:
sentence_generate(prob_dist_unigram_trump,prob_dist_bigram_trump,prob_dist_trigram_trump,3,\
                         prob_dist_trigram_trump,prob_dist_bigram_trump,prob_dist_unigram_trump)

NameError: name 'sentence_generate' is not defined

In [ ]:
l = [1,2,3,4,3,2,334234,2,2,3,3,4,4,3,2,2,2,2]

In [30]:
antagonist

"it.''that"

In [106]:
story_one = place_of_story(1,protagonist,antagonist,place_one,place_two, sentence_generate(prob_dist_unigram_trump,prob_dist_bigram_trump,prob_dist_trigram_trump,3,\
                         prob_dist_trigram_trump,prob_dist_bigram_trump,prob_dist_unigram_trump,text_counter_bigram_trump,text_counter_trigram_trump)) #protagonist
story_one

/Users/jonathanhilgart/DSCI6004-student/Final_Project/sentence_generator.py:55: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  generated_text.append(starting_chars[start_char_index])


['You have the feeling that something bad was going to happen so you pull out your pocketknife. alice looks at you and asks,"So what do you\t\tthink you are going to do with that? That is not going to save you. You notice cheshire in the distance and you yell to trump "Run for it!"',
 'All of a sudden, you remember that you have not eaten lunch. "Silly me!" you think to yourself as your pull out your half-eaten-sandwich.\t\tYou chew happily for a while before deciding it was time to more on. "Time to go you mention to Trump". As you walk you suddenly come across gryphon \t\twho asks "What are ye doing here?" ',
 ' After talking with alice you asked "What do we do around here?". alice replies, "Currently, I am avoiding gryphon because they are looking\t\tto round up everyone who speaks gryphon!" "Oh no!" You think to yourself "They are going to capture Trump!"Trump turns and says Come second chance and the other day on television , there will be by both parties — democrats , i promise .

In [112]:
query = ['Dam Trump!']

In [108]:
tfidf = TfidfVectorizer()

In [110]:
tf_idf_story= tfidf.fit_transform(story_one)

In [111]:
tf_idf_story

<3x126 sparse matrix of type '<class 'numpy.float64'>'
	with 166 stored elements in Compressed Sparse Row format>

In [113]:
query_tfidf = tfidf.transform(query)
query_tfidf

<1x126 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [114]:
cosine_sim = linear_kernel(tf_idf_story,query_tfidf).flatten()
cosine_sim

array([ 0.07034936,  0.07164993,  0.09629021])

In [115]:
cosine_sim.argsort()[::-1][0]

2

In [116]:
story_one[cosine_sim.argsort()[::-1][0]]

' After talking with alice you asked "What do we do around here?". alice replies, "Currently, I am avoiding gryphon because they are looking\t\tto round up everyone who speaks gryphon!" "Oh no!" You think to yourself "They are going to capture Trump!"Trump turns and says Come second chance and the other day on television , there will be by both parties — democrats , i promise . But they don’t know . Obamacare is a fact , they just bought it very , very strongly , you have to say , or to the back of the deal was a terrible , horrible , corrupt pacs – they’re corrupt – they don’t even know what’s going on in chicago .  .\t\tWould should you do next?'